# 4.3 インストラクションチューニングによる質問応答のファインチューニング（学習コード）

本Notebookでは、インストラクションチューニングによって文章生成AIをファインチューニングするコードを記載します。  
モデルの評価については、「3-1_InstructionTuning.ipynb」をご確認ください。

## 事前準備



### (1) ライブラリのインストール

必要なライブラリをインストールします。

In [ ]:
!pip install transformers==4.34.0
!pip install accelerate==0.23.0
!pip install datasets==2.15.0
!pip install peft==0.7.0
# !pip install deepspeed==0.12.4 # 一部依存関係でエラーが出るためコメントアウトしています。以降のコード中では不要です。
!pip install trl==0.7.4
!pip install bitsandbytes==0.41.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 47.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/82

In [ ]:
import warnings
warnings.simplefilter("ignore")

### (2) データセットの準備

必要なデータセットを準備します。  
ここでは、学習時間を短くするために学習データのサイズを絞っています。

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset


# JCommonsenseQAをダウンロードし、データフレームに変換する
dataset_qa = load_dataset("shunk031/JGLUE", name="JCommonsenseQA")
train_df_qa = dataset_qa["train"].to_pandas()
test_df_qa = dataset_qa["validation"].to_pandas()
qa_df = pd.concat([train_df_qa, test_df_qa])

# llm-japanese-dataset-valnillaをダウンロードし、データフレーム変換する
dataset_llm_japanese = load_dataset("izumi-lab/llm-japanese-dataset-vanilla", revision="1.0.1")
train_df = dataset_llm_japanese["train"].to_pandas()

# JCommonsenseQAのデータを除外する
qa_df["label_text"] = qa_df.apply(lambda x: x[f"choice{x['label']}"], axis=1)
mask = train_df["output"].isin(qa_df["label_text"]) & train_df["instruction"].isin(qa_df["question"])
train_df["is_JCommonsenseQA"] = mask
train_df = train_df[~train_df["is_JCommonsenseQA"]]
del train_df["is_JCommonsenseQA"]
print(f"除外したデータ数 : {sum(mask)}")

### InstructionTuning用のデータセットを作成する ###
# JCommonsenseQAデータセットの整備
qa_instruction_text = "質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。"
train_df_qa["instruction"] = qa_instruction_text
train_df_qa["input"] = train_df_qa.apply(lambda x: f"質問:\n{x['question']}\n選択肢:\n0.{x['choice0']} 1.{x['choice1']} 2.{x['choice2']} 3.{x['choice3']} 4.{x['choice4']}", axis=1)
train_df_qa["output"] = train_df_qa["label"].astype(str)
train_df_qa = train_df_qa[["instruction", "input", "output"]]

# 短時間で学習可能なサイズにサンプリングする
total_sample_size = 30000
sample_size = total_sample_size - len(train_df_qa)
train_df = train_df.sample(n=sample_size, random_state=42)

# train_dfと結合する
train_df = pd.concat([train_df, train_df_qa], ignore_index=True)

# datasetクラスに変換する
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(train_size=0.9, seed=42)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

除外したデータ数 : 10059


### (3) モデルの読み込み

使用するモデルを読み込みます。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# トークナイザ・モデルを読み込みます
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-13b-v1.0", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "llm-jp/llm-jp-13b-v1.0",
    load_in_8bit=True, # 計算を効率化させるため、重みをint8として読み込みます。
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## ファインチューニングの実装

学習前の出力確認用

In [ ]:
from transformers import pipeline


# パイプラインの構築
qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

prompt = """
### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
"""

generate_text = qa_pipeline(
    prompt,
    max_length=100,
    num_return_sequences=1,
    temperature=0
)[0]["generated_text"]
print(generate_text)


### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
富士山

### 出力:
正解です。

### 解説:
富士山は、日本の最高峰で、標高3776mです。

## 問題10

### 問題:
以下のような、


### (1) LoRAコンフィグ設定

学習に使用するコンフィグを設定します。

In [ ]:
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
      param.data = param.data.to(torch.float32)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

peft_config = LoraConfig(
    r=8,
    target_modules=["c_attn", "c_proj", "c_fc"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.CAUSAL_LM
)

training_arguments = TrainingArguments(
    output_dir="./training_logs",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50
)

[2024-04-15 08:21:15,991] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


### (2) Data Collator

Data Collatorを定義します。

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"### 指示:\n{example['instruction'][i]}\n\n### 入力:\n{example['input'][i]}\n\n### 回答:\n{example['output'][i]}"
        output_texts.append(text)
    return output_texts

collator = DataCollatorForCompletionOnlyLM(
        response_template="回答:\n", tokenizer=tokenizer
)

### (3) 学習の実行

学習を実行します。

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_arguments,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=2048
)

trainer.train()

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
50,1.280800,1.010575
100,1.048700,0.941168
150,1.051900,0.910835
200,1.048600,0.804850
250,0.995900,0.735657
300,0.999600,0.734285
350,1.024500,0.715881
400,0.981800,0.716565
450,0.999200,0.717991


Buffered data was truncated after reaching the output size limit.

In [ ]:
trainer.save_model() # モデルの保存
trainer.save_state() # メトリクスの保存